In [1]:
from datasets import load_from_disk
import numpy as np
import soundfile as sf
import os, sys

from sklearn.model_selection import train_test_split

SEED=42
SAMPLE_RATE=16000

ds = load_from_disk("../data/datasets/ds_462700")

/home/pierre/.local/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(ds)
print(f"Features: {ds.features}")
print(f"Number of samples: {len(ds)}")

Dataset({
    features: ['audio', 'label', 'sampling_rate'],
    num_rows: 462701
})
Features: {'audio': List(Value('float32')), 'label': ClassLabel(names=['other', 'drone']), 'sampling_rate': Value('int64')}
Number of samples: 462701


In [23]:
df = ds.to_pandas()
# Split into train (70%), temporary(30%)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=SEED, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED, stratify=temp_df['label'])

In [24]:
print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))
print("\nClass distribution in train:")
print(train_df['label'].value_counts())
print("\nClass distribution in validation:")
print(val_df['label'].value_counts())
print("\nClass distribution in test:")
print(test_df['label'].value_counts())

Train size: 134239
Validation size: 28765
Test size: 28766

Class distribution in train:
label
1    118662
0     15577
Name: count, dtype: int64

Class distribution in validation:
label
1    25427
0     3338
Name: count, dtype: int64

Class distribution in test:
label
1    25428
0     3338
Name: count, dtype: int64


In [37]:
import soundfile as sf

# Load a sample audio file
sample = ds[60000]
sampling_rate = sample['audio']['sampling_rate']
audio_array = sample['audio']['array']

print("Sampling rate:", sampling_rate)
print("Audio shape:", audio_array.shape)
print("Duration (seconds):", len(audio_array) / sampling_rate)
print("Label: ", sample['label'])

from IPython.display import Audio as DisplayAudio
display(DisplayAudio(audio_array, rate=sampling_rate))

Sampling rate: 16000
Audio shape: (8000,)
Duration (seconds): 0.5
Label:  1


In [55]:
# model_train_drone_audio.py
import os
import random
from collections import Counter
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from datasets import load_from_disk
from tqdm import tqdm

# optional metrics
try:
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
except Exception:
    accuracy_score = precision_score = recall_score = f1_score = None

# -------------------------
# Config / hyperparams
# -------------------------
DATA_PATH = "../data/datasets/ds_462700"   # your saved chunked dataset
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 750
LR = 3e-4
EPOCHS = 10
NUM_WORKERS = 24
SEED = 42
SAMPLE_RATE = 16000
CHUNK_SAMPLES = int(0.5 * SAMPLE_RATE)   # 8000
NUM_CLASSES = 2

torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)


# -------------------------
# Load & split HF dataset
# -------------------------
ds = load_from_disk(DATA_PATH)   # should be a single Dataset saved earlier
ds = ds.shuffle(seed=SEED)

# split: 80/10/10 (train/val/test)
tmp = ds.train_test_split(test_size=0.2, seed=SEED)
train_valid = tmp["train"]
test_ds = tmp["test"]
tv = train_valid.train_test_split(test_size=0.125, seed=SEED)  # 0.125 of 80 = 10%
train_ds = tv["train"]
valid_ds = tv["test"]

print("sizes:", len(train_ds), len(valid_ds), len(test_ds))


# -------------------------
# Lightweight augmentations (on-the-fly)
# -------------------------
def random_gain(wave, min_db=-6.0, max_db=6.0):
    g = 10 ** (random.uniform(min_db, max_db) / 20.0)
    return wave * g

def add_noise_snr(wave, min_snr_db=10, max_snr_db=30):
    snr_db = random.uniform(min_snr_db, max_snr_db)
    rms = torch.sqrt(torch.mean(wave ** 2) + 1e-9)
    noise_rms = rms / (10 ** (snr_db / 20.0))
    noise = torch.randn_like(wave) * noise_rms
    return wave + noise

def random_shift(wave, max_frac=0.1):
    # circular-ish shift with zero padding
    max_shift = int(wave.shape[-1] * max_frac)
    if max_shift == 0:
        return wave
    shift = random.randint(-max_shift, max_shift)
    if shift > 0:
        return torch.cat([wave[..., shift:], torch.zeros_like(wave[..., :shift])], dim=-1)
    elif shift < 0:
        s = -shift
        return torch.cat([torch.zeros_like(wave[..., -s:]), wave[..., :-s]], dim=-1)
    return wave

def train_augment(wave):
    # wave: tensor shape [1, L]
    if random.random() < 0.5:
        wave = random_gain(wave, -4, 4)
    if random.random() < 0.3:
        wave = add_noise_snr(wave, 10, 25)
    if random.random() < 0.3:
        wave = random_shift(wave, 0.08)
    return wave


# -------------------------
# PyTorch Dataset wrapper
# -------------------------
class HFAudioDataset(Dataset):
    def __init__(self, hf_dataset, augment=False):
        self.ds = hf_dataset
        self.augment = augment

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        # item['audio'] may be numpy array
        x = torch.tensor(item["audio"]).float()
        # ensure shape [1, L]
        if x.dim() == 1:
            x = x.unsqueeze(0)
        # simple per-sample peak normalization
        peak = x.abs().max().clamp_min(1e-9)
        x = x / peak
        if self.augment:
            x = train_augment(x)
        y = int(item["label"])
        return x, y


# -------------------------
# DataLoaders (balanced sampler)
# -------------------------
train_dataset = HFAudioDataset(train_ds, augment=True)
valid_dataset = HFAudioDataset(valid_ds, augment=False)
test_dataset = HFAudioDataset(test_ds, augment=False)

# build weights for WeightedRandomSampler to balance classes
train_labels = train_ds["label"]
counts = Counter(train_labels)
class_weights = {c: 1.0 / counts[c] for c in counts}
sample_weights = [class_weights[int(lbl)] for lbl in train_labels]

sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler,
                          num_workers=NUM_WORKERS, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                         num_workers=NUM_WORKERS, pin_memory=True)


# -------------------------
# Model (1D CNN)
# -------------------------
class ConvBlock1D(nn.Module):
    def __init__(self, in_ch, out_ch, kernel=7, stride=1, pool=True):
        super().__init__()
        padding = kernel // 2
        layers = [
            nn.Conv1d(in_ch, out_ch, kernel, stride=stride, padding=padding, bias=False),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True)
        ]
        if pool:
            layers.append(nn.MaxPool1d(2))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class Drone1DCNN(nn.Module):
    def __init__(self, n_classes=2, in_ch=1, base=32):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv1d(in_ch, base, kernel_size=11, stride=2, padding=5, bias=False),
            nn.BatchNorm1d(base),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(2)
        )
        self.b1 = ConvBlock1D(base, base * 2, kernel=7, stride=1, pool=True)   # /2
        self.b2 = ConvBlock1D(base * 2, base * 4, kernel=5, stride=1, pool=True)  # /2
        self.b3 = ConvBlock1D(base * 4, base * 8, kernel=3, stride=1, pool=True)  # /2
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(base * 8, n_classes)

    def forward(self, x):
        # x: [B, 1, L]
        x = self.stem(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.global_pool(x).squeeze(-1)   # [B, C]
        x = self.fc(x)
        return x

model = Drone1DCNN(n_classes=NUM_CLASSES, in_ch=1, base=32).to(DEVICE)
print("model params:", sum(p.numel() for p in model.parameters() if p.requires_grad))


# -------------------------
# Training utils
# -------------------------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
# simple scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=2)

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE == "cuda"))

def evaluate(model, loader):
    model.eval()
    losses = []
    preds = []
    targets = []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
                logits = model(xb)
                loss = criterion(logits, yb)
            losses.append(loss.item() * xb.size(0))
            p = logits.argmax(dim=1).cpu().numpy()
            preds.extend(p.tolist())
            targets.extend(yb.cpu().numpy().tolist())
    loss = sum(losses) / max(1, len(loader.dataset))
    acc = None
    prec = rec = f1 = None
    if accuracy_score:
        acc = accuracy_score(targets, preds)
        prec = precision_score(targets, preds, zero_division=0)
        rec = recall_score(targets, preds, zero_division=0)
        f1 = f1_score(targets, preds, zero_division=0)
    return {"loss": loss, "acc": acc, "prec": prec, "rec": rec, "f1": f1}


# -------------------------
# Training loop
# -------------------------
best_val_f1 = -1.0
checkpoint_dir = Path("checkpoints")
checkpoint_dir.mkdir(exist_ok=True)

for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False)
    for xb, yb in pbar:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
            logits = model(xb)
            loss = criterion(logits, yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * xb.size(0)
        pbar.set_postfix(loss=loss.item())

    train_loss = running_loss / len(train_loader.dataset)
    val_metrics = evaluate(model, valid_loader)
    val_f1 = val_metrics["f1"] if val_metrics["f1"] is not None else -1.0

    scheduler.step(val_f1)

    print(f"Epoch {epoch} | train_loss={train_loss:.4f} val_loss={val_metrics['loss']:.4f} val_f1={val_metrics['f1']}")
    # save best
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save({"model_state": model.state_dict(),
                    "optim_state": optimizer.state_dict(),
                    "epoch": epoch},
                   checkpoint_dir / "best.pth")
        print("Saved best checkpoint.")


# -------------------------
# Final test metrics
# -------------------------
ckpt = torch.load(checkpoint_dir / "best.pth", map_location=DEVICE)
model.load_state_dict(ckpt["model_state"])
test_metrics = evaluate(model, test_loader)
print("TEST:", test_metrics)



sizes: 323890 46270 92541


/tmp/ipykernel_46319/279987515.py:200: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE == "cuda"))


model params: 155426


Epoch 1/10:   0%|          | 0/432 [00:00<?, ?it/s]/tmp/ipykernel_46319/279987515.py:244: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
/tmp/ipykernel_46319/279987515.py:211: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):


Epoch 1 | train_loss=0.1802 val_loss=0.1215 val_f1=0.9518695306284806
Saved best checkpoint.


...

Epoch 9 | train_loss=0.0873 val_loss=0.0725 val_f1=0.9729097071517397
Saved best checkpoint.


Epoch 10/10:   0%|          | 0/432 [00:00<?, ?it/s]/tmp/ipykernel_46319/279987515.py:244: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
/tmp/ipykernel_46319/279987515.py:211: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):


Epoch 10 | train_loss=0.0861 val_loss=0.1218 val_f1=0.9503861455651766


/tmp/ipykernel_46319/279987515.py:211: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):


TEST: {'loss': 0.0726263311347065, 'acc': 0.9759674090403173, 'prec': 0.9787358137500304, 'rec': 0.9675900343560051, 'f1': 0.973131010486638}


In [45]:
# -------------------------
# Inference: sliding-window for long audio (example)
# -------------------------
def infer_long_audio(wave_np, model, window_size=CHUNK_SAMPLES, hop_size=CHUNK_SAMPLES//2, device=DEVICE):
    """
    wave_np: 1D numpy array (raw waveform), model expects normalized [-1,1] per chunk
    returns: averaged softmax probabilities across windows
    """
    model.eval()
    x = torch.tensor(wave_np).float()
    # pad to multiple
    rem = x.shape[-1] % window_size
    if rem != 0:
        pad = window_size - rem
        x = torch.nn.functional.pad(x, (0, pad))
    probs = []
    with torch.no_grad():
        for i in range(0, x.shape[-1] - window_size + 1, hop_size):
            chunk = x[i:i+window_size].unsqueeze(0).unsqueeze(0)  # [1,1,L]
            peak = chunk.abs().max().clamp_min(1e-9)
            chunk = chunk / peak
            chunk = chunk.to(device)
            logits = model(chunk)
            p = torch.softmax(logits, dim=1)[0].cpu().numpy()
            print(p)
            probs.append(p)
    if not probs:
        return None
    probs = np.stack(probs, axis=0)
    avg = probs.mean(axis=0)
    return avg  # vector of class probabilities

# Example usage:
import soundfile as sf
import torchaudio

# wave, sr = sf.read("../data/raw/dataset2/other/loutput_000000000.wav")
# wave, sr = sf.read("../data/raw/hibou_dataset/drone/2949-2949.wav")
# wave, sr = sf.read("/home/pierre/Documents/Projects/PST4/AI/data/raw/recordings/03/9.wav")
wave, sr = sf.read("/home/pierre/Downloads/B_S2_D1_092-bebop_000_.wav")

if sr != SAMPLE_RATE:
    wave_t = torch.tensor(wave).float()
    wave_t = torchaudio.functional.resample(wave_t, orig_freq=sr, new_freq=SAMPLE_RATE)
    wave = wave_t.numpy()
    sr = SAMPLE_RATE

probs = infer_long_audio(wave, model)
is_drone = probs[1] > 0.5   # or choose a tuned threshold
print(f"Drone detected? {is_drone}, prob={probs[1]:.3f}")


[0.9068652 0.0931348]
[0.87450904 0.12549102]
[0.9413931  0.05860687]
[0.8820874  0.11791257]
[0.7927327  0.20726728]
Drone detected? False, prob=0.120


In [50]:
# train_drone_sklearn.py
import numpy as np
import soundfile as sf
from pathlib import Path
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import librosa

# Config
SAMPLE_RATE = 16000
CHUNK_SEC = 0.5
CHUNK_SAMPLES = int(CHUNK_SEC * SAMPLE_RATE)

# 1. Load all audio and labels
X, y = [], []
root = Path("../data/datasets/ds_462700")  # adjust path
for label_name in ["other", "drone"]:
    for wav_path in (root / label_name).glob("*.wav"):
        wave, sr = sf.read(wav_path)
        if sr != SAMPLE_RATE:
            wave = librosa.resample(wave, orig_sr=sr, target_sr=SAMPLE_RATE)
        # Split into fixed chunks
        for i in range(0, len(wave) - CHUNK_SAMPLES + 1, CHUNK_SAMPLES):
            chunk = wave[i:i+CHUNK_SAMPLES]
            # Extract log-mel features
            mel = librosa.feature.melspectrogram(
                y=chunk, sr=SAMPLE_RATE, n_mels=64, n_fft=1024, hop_length=512
            )
            logmel = librosa.power_to_db(mel, ref=np.max)
            feat = logmel.mean(axis=1)  # average over time -> 64-dim feature
            X.append(feat)
            y.append(1 if label_name == "drone" else 0)

X, y = np.stack(X), np.array(y)

# 2. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Train classifier
clf = GradientBoostingClassifier(n_estimators=300, max_depth=3)
clf.fit(X_train, y_train)

# 5. Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=["other", "drone"]))

# 6. Save model + scaler
dump({"model": clf, "scaler": scaler}, "drone_clf.joblib")


ValueError: need at least one array to stack

In [53]:
import os
import numpy as np
import tensorflow as tf
from datasets import load_dataset
from scipy.signal import spectrogram

# --- Load dataset ---
ds = load_from_disk("../data/datasets/ds_462700")

# Assume ds has 'audio' field with file paths and 'label' field
def load_audio(sample):
    audio = tf.io.read_file(sample['audio'])
    waveform, _ = tf.audio.decode_wav(audio, desired_channels=1)
    waveform = tf.squeeze(waveform, axis=-1)  # mono
    return {"waveform": waveform, "label": sample['label']}

ds = ds.map(load_audio)

# --- Generate spectrograms ---
def make_spectrogram(sample, nperseg=256, noverlap=128):
    f, t, Sxx = spectrogram(sample['waveform'].numpy(), fs=48000, nperseg=nperseg, noverlap=noverlap)
    Sxx = np.log1p(Sxx)  # log scale
    Sxx = Sxx.astype(np.float32)
    return {"spec": Sxx[..., np.newaxis], "label": sample['label']}

specs = [make_spectrogram(s) for s in ds]

X = np.array([s['spec'] for s in specs])
y = np.array([s['label'] for s in specs])
y = tf.keras.utils.to_categorical(y, num_classes=2)

# --- Simple CNN ---
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=X.shape[1:]),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# --- Train/test split ---
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=16, callbacks=[early_stop])


Map:   0%|          | 0/462701 [00:00<?, ? examples/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1758645591.555046   46319 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1735 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
Map:   0%|          | 0/462701 [00:00<?, ? examples/s]


TypeError: Cannot convert [-0.004547119140625, -0.00469970703125, -0.004913330078125, -0.003814697265625, -0.001251220703125, 0.0013427734375, 0.0032958984375, 0.004669189453125, 0.004364013671875, 0.003662109375, 0.002197265625, 0.0010986328125, 0.00042724609375, 0.0006103515625, 0.001068115234375, 0.00244140625, 0.0028076171875, 0.003204345703125, 0.00335693359375, 0.0028076171875, 0.002166748046875, 0.001007080078125, -0.000885009765625, -0.002227783203125, -0.0023193359375, -0.001617431640625, -0.000335693359375, 0.000274658203125, 0.000732421875, 0.00079345703125, -0.000152587890625, -0.001495361328125, -0.002838134765625, -0.00421142578125, -0.00482177734375, -0.00494384765625, -0.00518798828125, -0.0040283203125, -0.001129150390625, 0.001556396484375, 0.00311279296875, 0.00347900390625, 0.003021240234375, 0.001678466796875, 0.000762939453125, 0.0006103515625, 0.000946044921875, 0.001373291015625, 0.002410888671875, 0.002685546875, 0.002197265625, 0.0015869140625, 0.0015258789062